In [25]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import hog
from tqdm.notebook import tqdm

In [24]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img

In [26]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [27]:
def createdataframe(dir):
    image_paths = [] # List to store paths of images
    labels = [] # List to store paths of images

    # Loop through each label (subdirectory) in the given directory
    for label in os.listdir(dir):
        # Loop through each image in the label's subdirectory
        for imagename in os.listdir(os.path.join(dir,label)):
            # Construct the full path to the image and add it to the image_paths list
            image_paths.append(os.path.join(dir,label,imagename))
             # Add the label corresponding to the image to the labels list
            labels.append(label)
        print(label, "completed") # Print the completion status for the label
    return image_paths,labels # Return the lists of image paths and labels

In [28]:
train = pd.DataFrame() 
train['image'], train['label'] = createdataframe(TRAIN_DIR) # Populate the DataFrame with image paths and labels

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [29]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [30]:
# HOG Feature Extraction Function
def extract_hog_features(images):
    hog_features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale", target_size=(48, 48))
        img = np.array(img)
        features, _ = hog(img, orientations=8, pixels_per_cell=(8, 8), 
                          cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
        hog_features.append(features)
    return np.array(hog_features)

In [31]:
# Extract HOG features for train and test sets
x_train_hog = extract_hog_features(train['image'])
x_test_hog = extract_hog_features(test['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

  0%|          | 0/7178 [00:00<?, ?it/s]

In [32]:
# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(train['label'])
y_test_encoded = le.transform(test['label'])

In [ ]:
# Train an SVM model
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(x_train_hog, y_train_encoded)

In [ ]:
# Predict on the test set
y_pred = svm_model.predict(x_test_hog)

In [ ]:
# Calculate Metrics
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_test_encoded, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Accuracy Plot
plt.figure(figsize=(6, 4))
plt.bar(['SVM Test Accuracy'], [accuracy], color='skyblue')
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.show()